In [118]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="processed_severity_data.csv")["train"]
print(dataset)

dataset = dataset.train_test_split(test_size=0.2)  # 80% train, 20% test

train_dataset = dataset["train"]
test_dataset = dataset["test"]

train_dataset = train_dataset.rename_column("attention_masks", "mask")
test_dataset = test_dataset.rename_column("attention_masks", "mask")

train_dataset = train_dataset.rename_column("input_ids", "ids")
test_dataset = test_dataset.rename_column("input_ids", "ids")

# train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
# test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


print(train_dataset[0])

Dataset({
    features: ['input_ids', 'attention_masks', 'labels'],
    num_rows: 4402
})
{'ids': '[101, 8827, 15459, 13102, 18491, 3593, 1024, 25801, 2692, 28154, 18888, 1013, 4531, 1024, 1996, 3424, 1011, 7540, 2723, 2013, 7861, 2121, 2290, 1012, 9602, 2597, 2000, 6164, 2341, 2025, 3024, 1012, 6412, 19184, 1024, 1996, 3395, 6019, 2930, 2003, 2761, 3024, 2007, 3424, 1011, 8301, 2094, 6773, 1012, 1999, 1996, 3522, 8774, 1010, 2004, 2112, 1997, 9410, 25536, 6032, 1010, 1996, 9602, 6718, 2282, 5877, 2001, 4993, 1998, 13967, 9100, 1012, 2023, 2435, 1996, 2878, 5877, 1037, 2978, 1997, 1037, 19504, 2298, 1012, 2174, 1010, 1996, 19467, 1010, 4993, 2007, 3424, 1011, 8301, 2094, 6773, 2761, 1010, 2001, 2145, 2200, 2172, 6464, 4346, 3424, 1011, 8301, 2094, 5144, 1012, 3040, 11548, 2023, 2000, 1996, 8827, 3597, 1010, 2174, 2002, 2106, 2025, 5993, 1012, 3040, 2515, 2025, 102]', 'mask': '[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [119]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [120]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [121]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [122]:
# class CustomDataset(Dataset):

#     def __init__(self, dataframe, tokenizer, max_len):
#         self.tokenizer = tokenizer
#         self.data = dataframe
#         self.comment_text = dataframe.comment_text
#         self.targets = self.data.list
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.comment_text)

#     def __getitem__(self, index):
#         comment_text = str(self.comment_text[index])
#         comment_text = " ".join(comment_text.split())

#         inputs = self.tokenizer.encode_plus(
#             comment_text,
#             None,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             pad_to_max_length=True,
#             return_token_type_ids=True
#         )
#         ids = inputs['input_ids']
#         mask = inputs['attention_mask']
#         labels = inputs["labels"]


#         return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long),
#             'labels': torch.tensor(labels, dtype=torch.long),
#             'targets': torch.tensor(self.targets[index], dtype=torch.float)
#         }

In [123]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(train_dataset, **train_params)
testing_loader = DataLoader(test_dataset, **test_params)

In [124]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [125]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [126]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [129]:
def train(epoch):
    model.train()
    for _, data in enumerate(training_loader, 0):
        print(data['ids'])
        ids = torch.tensor(data['ids']).to(device, dtype=torch.long)
        mask = torch.tensor(data['mask']).to(device, dtype=torch.long)
        token_type_ids = torch.tensor(data['label']).to(device, dtype=torch.long)
        targets = torch.tensor(data['targets']).to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [130]:
for epoch in range(EPOCHS):
    train(epoch)

['[101, 8827, 15459, 13102, 18491, 3593, 1024, 21417, 21619, 2509, 18888, 1013, 4531, 1024, 4812, 2044, 4926, 6412, 19184, 1024, 2006, 2251, 2382, 1010, 16798, 2509, 1010, 11113, 6819, 14220, 7303, 2000, 2022, 2207, 2013, 2010, 2851, 3422, 2012, 1019, 1024, 4002, 2572, 2044, 7678, 3155, 2028, 3178, 1997, 1996, 2958, 3422, 1012, 2004, 2009, 2001, 2525, 11695, 1010, 1996, 4611, 2961, 3039, 2032, 2000, 2175, 2091, 1012, 1996, 6258, 2001, 2012, 2712, 1999, 2330, 5380, 1010, 4372, 22494, 2618, 2000, 2710, 1012, 2012, 2105, 5709, 11387, 8093, 2015, 2045, 2001, 2019, 11240, 5043, 5994, 1996, 11113, 6819, 14220, 1012, 11240, 2597, 1024, 4229, 3207, 2290, 2484, 1012, 1021, 1005, 1050, 1013, 11502, 2139, 2290, 2676, 1012, 1015, 1005, 1041, 3058, 1004, 2051, 1024, 13293, 21650, 16798, 102]', '[101, 8827, 15459, 13102, 18491, 3593, 1024, 14112, 27814, 2620, 18888, 1013, 4531, 1024, 3417, 1011, 2217, 11240, 2166, 8569, 6977, 2003, 2025, 3201, 2000, 2022, 6908, 2012, 2712, 1006, 1996, 2713, 9231, 20

KeyError: 'input_ids'

In [62]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            print(data["input_ids"])
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [63]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

['[101, 8827, 15459, 13102, 18491, 3593, 1024, 12169, 2581, 23833, 18888, 1013, 4531, 1024, 2557, 6046, 5997, 7127, 1024, 6046, 8080, 1999, 2224, 1012, 3626, 2020, 2025, 5204, 2055, 2009, 1012, 6412, 19184, 1024, 2025, 2825, 2000, 2224, 13938, 5104, 2015, 3941, 1999, 2553, 1997, 5057, 2349, 2000, 2659, 3977, 1997, 5057, 10274, 1012, 6234, 5320, 1024, 11339, 3738, 6439, 12407, 2006, 6046, 5997, 1012, 3738, 2025, 7919, 5220, 3550, 2007, 13938, 5104, 2015, 3941, 1998, 4216, 1997, 2373, 4425, 7117, 3426, 4106, 1024, 11339, 3738, 6439, 12407, 2006, 6046, 5997, 1012, 3738, 2025, 7919, 5220, 3550, 2007, 13938, 5104, 2015, 3941, 1998, 4216, 1997, 2373, 4425, 6149, 3512, 2895, 1024, 3625, 2005, 13938, 5104, 2015, 3941, 2961, 2001, 10290, 2000, 9975, 3582, 9095, 1997, 3679, 1010, 102]', '[101, 8827, 15459, 13102, 18491, 3593, 1024, 12651, 8889, 2581, 18888, 1013, 4531, 1024, 3082, 3033, 8250, 2006, 5877, 2648, 11366, 2200, 3532, 1013, 5410, 25210, 2075, 2109, 1012, 2025, 5123, 2013, 2929, 1012, 

KeyError: 'ids'